The recipe for today is just doing One-Hot most columns and Thermometer encoding of some ordinal columns. Finally, we apply plain Logistic Regression.

Did anyone have success with more complex methods? Please let us know!

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import scipy
from tqdm import tqdm_notebook as tqdm

In [2]:
dd0=pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")
ddtest0=pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")
ddall=dd0.append(ddtest0, sort=False)
num_train=len(dd0)
ddall.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0.0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0.0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0.0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1.0
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0.0


# Crazy feature engineering

In [3]:
drop_cols=["bin_0"]

# Split 2 Letters; This is the only part which is not generic and would actually require data inspection
ddall["ord_5a"]=ddall["ord_5"].str[0]
ddall["ord_5b"]=ddall["ord_5"].str[1]
drop_cols.append("ord_5")

## Map cat vals which are not in both sets to single values

In [4]:
for col in ["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"]:
    train_vals = set(dd0[col].unique())
    test_vals = set(ddtest0[col].unique())
   
    xor_cat_vals=train_vals ^ test_vals
    if xor_cat_vals:
        ddall.loc[ddall[col].isin(xor_cat_vals), col]="xor"

## One-Hot Encode all

In [5]:
X=ddall[ddall.columns.difference(["id", "target"] + drop_cols)]

In [6]:
X_oh=X[X.columns.difference(["ord_1", "ord_4", "ord_5a", "ord_5b", "day", "month"])]
oh1=pd.get_dummies(X_oh, columns=X_oh.columns, drop_first=True, sparse=True)
ohc1=oh1.sparse.to_coo()

## Thermometer encode some ordinal columns

In [7]:
from sklearn.base import TransformerMixin
from itertools import repeat
import scipy


class ThermometerEncoder(TransformerMixin):
    """
    Assumes all values are known at fit
    """
    def __init__(self, sort_key=None):
        self.sort_key = sort_key
        self.value_map_ = None
    
    def fit(self, X, y=None):
        self.value_map_ = {val: i for i, val in enumerate(sorted(X.unique(), key=self.sort_key))}
        return self
    
    def transform(self, X, y=None):
        values = X.map(self.value_map_)
        
        possible_values = sorted(self.value_map_.values())
        
        idx1 = []
        idx2 = []
        
        all_indices = np.arange(len(X))
        
        for idx, val in enumerate(possible_values[:-1]):
            new_idxs = all_indices[values > val]
            idx1.extend(new_idxs)
            idx2.extend(repeat(idx, len(new_idxs)))
            
        result = scipy.sparse.coo_matrix(([1] * len(idx1), (idx1, idx2)), shape=(len(X), len(possible_values)), dtype="int8")
            
        return result

In [8]:
thermos=[]
for col in ["ord_1", "ord_2", "ord_3", "ord_4", "ord_5a", "day", "month"]:
    if col=="ord_1":
        sort_key=['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster'].index
    elif col=="ord_2":
        sort_key=['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot'].index
    elif col in ["ord_3", "ord_4", "ord_5a"]:
        sort_key=str
    elif col in ["day", "month"]:
        sort_key=int
    else:
        raise ValueError(col)
    
    enc=ThermometerEncoder(sort_key=sort_key)
    thermos.append(enc.fit_transform(X[col]))

## Combine sparse matrices

In [9]:
ohc=scipy.sparse.hstack([ohc1] + thermos).tocsr()
display(ohc)

X_train = ohc[:num_train]
X_test = ohc[num_train:]
y_train = dd0["target"].values

<500000x16091 sparse matrix of type '<class 'numpy.int16'>'
	with 34244885 stored elements in Compressed Sparse Row format>

# Make prediction

In [10]:
clf=LogisticRegression(C=0.123456789, solver="lbfgs", max_iter=5000)  # MODEL

clf.fit(X_train, y_train)

pred=clf.predict_proba(X_test)[:,1]

pd.DataFrame({"id": ddtest0["id"], "target": pred}).to_csv("submission.csv", index=False)

# Evaluate

In [11]:
from sklearn.model_selection import cross_validate

score=cross_validate(clf, X_train, y_train, cv=3, scoring="roc_auc")["test_score"].mean()
print(f"{score:.6f}")

0.802516
